In [37]:
from pymongo import MongoClient
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [2]:
conn = MongoClient("localhost:27017")

In [3]:
db = conn.get_database("ironhack")

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [5]:
#Vamos a empezar con la base de datos ironhack, con la collection companies. 
db.list_collection_names()

['countries_small', 'companies', 'books', 'restaurants']

In [6]:
#Nuestro punto de partida va a ser las localizaciones de las diferentes empresas de la base de datos, el resto de datos en principio nos es irrelevante.
#Aunque hay algunos que no lo son, como ahora veremos. 
collection = db.get_collection("companies")

In [7]:
#Veamos que tiene dentro la database de cada compañia.
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [8]:
#We are going to reduce the database based on 2 conditions:
    #Startups -> Founded after 2010
    #Successful -> Money raised so far > $1M
a = collection.find(
    {"$and": [
        {"founded_year": {"$gt": 2010}},
        {"total_money_raised": {"$regex": "M$"}},
        {"total_money_raised": {"$ne": "$0"}},
        {"offices.latitude": {"$ne": None}},
        {"offices.latitude": {"$exists": True}}
         ]},
        {"_id": 0, "name": 1, "category_code": 1, "founded_year": 1, "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1})

In [9]:
#Viendo el dataframe vemos como se ha reducido la lista a unas pocas empresas que cumplen las características
dfa = pd.DataFrame(a)
dfa

,name,category_code,founded_year,total_money_raised,offices
0,PeekYou,search,2012,$1.83M,"[{'latitude': 40.757929, 'longitude': -73.9855..."
1,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323..."
2,Widgetbox,other,2012,$14.5M,"[{'latitude': 37.798853, 'longitude': -122.398..."
3,RazorGator,ecommerce,2011,$58.8M,"[{'latitude': 34.047312, 'longitude': -118.445..."
4,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161..."
5,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32..."
6,Moblica,mobile,2011,$6M,"[{'latitude': 32.0554, 'longitude': 34.7595}]"
7,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789..."
8,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789..."
9,VisualOn,software,2011,$3.1M,"[{'latitude': 37.270518, 'longitude': -121.955..."


In [10]:
#Ahora necesitamos extraer el valor de la latitud y la longuitud de dentro de esa lista de diccionarios que tenemos en la columna offices. 
def get_values_as_list(dict_list, keys):
    list_ = [list(d[k] for k in keys) for d in dict_list]
    return list_[0]

In [11]:
#Ahora tenemos que extraer la latitude y longitud de las diferentes oficinas 
get_values_as_list(dfa.offices[0], ("latitude", "longitude"))

[40.757929, -73.985506]

In [12]:
def get_coordinates(Series, new_list):
    for i in Series.index:
        a = get_values_as_list(Series[i], ("latitude", "longitude"))
        new_list.append(a)
    return new_list
        

In [14]:
coordinates = []
dfa["coordinates"] = get_coordinates(dfa["offices"], coordinates)

In [24]:
latitudes = []
longitudes = []
for i in dfa["coordinates"]:
    latitudes.append(i[0])
    longitudes.append(i[1])

In [26]:
dfa["latitudes"] = latitudes
dfa["longitudes"] = longitudes

In [54]:
dfa.to_csv("Data/successful_startups_dataset.csv")

In [27]:
dfa

,name,category_code,founded_year,total_money_raised,offices,coordinates,latitudes,longitudes
0,PeekYou,search,2012,$1.83M,"[{'latitude': 40.757929, 'longitude': -73.9855...","[40.757929, -73.985506]",40.757929,-73.985506
1,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","[37.566879, -122.323895]",37.566879,-122.323895
2,Widgetbox,other,2012,$14.5M,"[{'latitude': 37.798853, 'longitude': -122.398...","[37.798853, -122.398599]",37.798853,-122.398599
3,RazorGator,ecommerce,2011,$58.8M,"[{'latitude': 34.047312, 'longitude': -118.445...","[34.047312, -118.445243]",34.047312,-118.445243
4,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","[37.446823, -122.161523]",37.446823,-122.161523
5,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","[37.5645375, -122.3254705]",37.564538,-122.325470
6,Moblica,mobile,2011,$6M,"[{'latitude': 32.0554, 'longitude': 34.7595}]","[32.0554, 34.7595]",32.055400,34.759500
7,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070
8,travelmob,travel,2011,$1M,"[{'latitude': 21.303049, 'longitude': -157.789...","[21.303049, -157.78907]",21.303049,-157.789070
9,VisualOn,software,2011,$3.1M,"[{'latitude': 37.270518, 'longitude': -121.955...","[37.270518, -121.955879]",37.270518,-121.955879


In [46]:
map_startups = Map(location=[21.303049, -157.789070], zoom_start=5)
map_startups

In [51]:
for i,row in dfa.iterrows():
    name = {
        "location" : [row["latitudes"], row["longitudes"]],
        "tooltip" : row["name"]
    }
    Marker(**name, icon=Icon(color = "green", prefix = "fa", icon="briefcase", icon_color = "black")).add_to(map_startups)

In [52]:
#Construimos un mapa inicial para ver los diferentes lugares que cumplirían con una de las características que estamos valorando. 
map_startups

In [53]:
heat_map_startups = Map(location=[40.757929,-73.985506],zoom_start=15)
HeatMap(data = dfa[["latitudes", "longitudes"]], radius = 15).add_to(heat_map_startups)
heat_map_startups